## SED stats

- Read SYNAGE++ results
- Output statistics about SED


In [1]:
import numpy as np
import shutil
from astropy.io import ascii, fits
import synCl
from prettytable import PrettyTable

syn = synCl.synClean()

syn = synCl.synClean()

rootDir = '/home/maccagni/Projects/MFS/FornaxA/contData/core/'

synDir = rootDir+'synOut/'
tabOut = rootDir+'outTabs/'

label = ['Beam','Out']
synMod = ['CI', 'CIOFF']
aut = ['Pbcorr']

syn.rootdir = rootDir


print '''\t+---------+\n\t Inputs loaded\n\t+---------+'''


	+---------+
	 Inputs loaded
	+---------+


In [ ]:
#-------------------------------------------------#
# SYNAGE stats
#-------------------------------------------------#

ciBeam = synDir+'core'+label[0]+aut[0]+synMod[0]+'.stats'
ciOffBeam = synDir+'core'+label[0]+aut[0]+synMod[1]+'.stats'
statsBeamCI = syn.readCIStats(ciBeam)
statsBeamCIOFF = syn.readCIOFFStats(ciOffBeam)

print '''\t+---------+\n\t Central Beam \n\t+---------+'''
CI = PrettyTable(['CI', 'value'])
for key, val in statsBeamCI.items():
    CI.add_row([key, val])
print CI
CIOFF = PrettyTable(['CIOFF', 'value'])
for key, val in statsBeamCIOFF.items():
    CIOFF.add_row([key, val])
print CIOFF


ciOut = synDir+'core'+label[1]+aut[0]+synMod[0]+'.stats'
ciOffOut = synDir+'core'+label[1]+aut[0]+synMod[1]+'.stats'
statsOutCI = syn.readCIStats(ciOut)
statsOutCIOFF = syn.readCIOFFStats(ciOffOut)


print '''\t+---------+\n\t Outer centre \n\t+---------+'''
CI = PrettyTable(['CI', 'value'])
for key, val in statsOutCI.items():
    CI.add_row([key, val])
print CI


CIOFF = PrettyTable(['CIOFF', 'value'])
for key, val in statsOutCIOFF.items():
    CIOFF.add_row([key, val])
print CIOFF


In [ ]:
#-------------------------------------------------#
# Different stats
#-------------------------------------------------#

D_L = 20.8*1e6*3.0857e16
CoreBeam= ascii.read(tabOut+'coreFlux'+label[0]+aut[0]+'.tbl', fast_reader=False)
SBeam =CoreBeam['Integrated Flux [Jy]'][1]
CoreOut= ascii.read(tabOut+'coreFlux'+label[1]+aut[0]+'.tbl', fast_reader=False)
SOut =CoreOut['Integrated Flux [Jy]'][1]

PCore= 4*np.pi*10e-26*np.power(D_L,2)*SBeam
POut = 4*np.pi*10e-26*np.power(D_L,2)*SOut

print '''\t+---------+\n\t Radio Power \n\t+---------+'''
print '''\n\tCore\tJets'''
print str(np.round(PCore,1))+'\t'+str(np.round(POut,1))+'\n'
print '''\tP jet/P core'''
print '\t'+str(np.round(POut/PCore,3))

lr = 101e-3
aBeam = 18.5/2.335
bBeam = 9./2.335
cBeam = aBeam/2.335

rOut = 8
hOut = 50.


VBeam = 4/3*np.pi*aBeam*bBeam*cBeam*np.power(lr,3)

VOut = np.pi*rOut*rOut*hOut*np.power(lr,3)

B_eqBeam =9.6e-12*np.power(2*PCore/VBeam,2./7.)
B_eqOut =9.6e-12*np.power(2*POut/VOut,2./7.)

print '''\n\t+---------+\n\t Equipartition Magnetic Field \n\t+---------+'''
print '''\n\tCore\tJets'''
print str(np.round(B_eqBeam,8))+'\t'+str(np.round(B_eqOut,8))+'\n'


U_eqBeam = np.power(B_eqBeam,2)*7./(24.*np.pi)
U_eqOut = np.power(B_eqOut,2)*7./(24.*np.pi)

print '''\n\t+---------+\n\t Minimum Energy density [erg cm-3 , dyne cm -2]\n\t+---------+'''
print '''\n\tCore\tJets'''
print str(np.round(U_eqBeam,18))+'\t'+str(np.round(U_eqOut,18))+'\n'

In [ ]:
B = np.array([B_eqBeam,B_eqOut])*1e6
z=0.00578
BIC = 3.25*np.power(1+z,2)

for i in xrange(0,len(label)):
    
    statsCI = syn.readCIStats(synDir+'core'+label[i]+aut[0]+'CI.stats')
    statsCIOFF =  syn.readCIOFFStats(synDir+'core'+label[i]+aut[0]+'CIOFF.stats')

    tsCI = 1610.*(np.power(B[i],0.5))/(B[i]*B[i]+BIC*BIC)*np.divide(1.,np.power(statsCI['break']/1e3*(1.+z),0.5))
    tsCIsup = 1610.*(np.power(B[i],0.5))/(B[i]*B[i]+BIC*BIC)*np.divide(1.,np.power(statsCI['break_sup']/1e3*(1.+z),0.5))
    tsCIinf = 1610.*(np.power(B[i],0.5))/(B[i]*B[i]+BIC*BIC)*np.divide(1.,np.power(-statsCI['break_inf']/1e3*(1.+z),0.5))

    statsCI['ts'] = tsCI
    statsCI['ts_sup'] = tsCIsup
    statsCI['ts_inf'] = tsCIinf
        
    tsCIOFF = 1610.*(np.power(B[i],0.5))/(B[i]*B[i]+BIC*BIC)*np.divide(1.,np.power(statsCIOFF['break']/1e3*(1.+z),0.5))
    tsCIOFFsup = 1610.*(np.power(B[i],0.5))/(B[i]*B[i]+BIC*BIC)*np.divide(1.,np.power(statsCI['break_sup']/1e3*(1.+z),0.5))
    tsCIOFFinf = 1610.*(np.power(B[i],0.5))/(B[i]*B[i]+BIC*BIC)*np.divide(1.,np.power(-statsCI['break_inf']/1e3*(1.+z),0.5))
    
    statsCIOFF['ts'] = tsCIOFF
    statsCIOFF['ts_sup'] = tsCIOFFsup
    statsCIOFF['ts_inf'] = tsCIOFFinf
    
    tOFF = statsCIOFF['tratio']*tsCIOFF
    statsCIOFF['toff'] = tOFF
    tOFFsup = statsCIOFF['t_errsup']*tsCIOFFsup
    statsCIOFF['toff_sup'] = tOFFsup
    tOFFinf = -statsCIOFF['t_errinf']*tsCIOFFsup
    statsCIOFF['toff_inf'] = tOFFinf

    if i ==0:
        statsBeamCI = statsCI.copy()
        statsBeamCIOFF = statsCIOFF.copy()      
    elif i ==1:
        statsOutCI = statsCI.copy()
        statsOutCIOFF = statsCIOFF.copy()      

print statsOutCIOFF
        
print '''\t+---------+\n\t Spectral Ages \n\t+---------+'''

print '\tCore  ts =\t'+str(np.round(statsBeamCI['ts'],6))+' + '+str(np.round(statsBeamCI['ts_sup'],6))+' - '+str(np.round(statsBeamCI['ts_inf'],6))
print '\tCoreO ts =\t'+str(np.round(statsOutCIOFF['ts'],6))+' + '+str(np.round(statsOutCIOFF['ts_sup'],6))+' - '+str(np.round(statsOutCIOFF['ts_inf'],6))
print '\tCoreO toff =\t'+str(np.round(statsOutCIOFF['toff'],6))+' + '+str(np.round(statsOutCIOFF['toff_sup'],8))+' - '+str(np.round(statsOutCIOFF['toff_inf'],6))
print '\tCoreO tratio =\t'+str(np.round(statsOutCIOFF['tratio'],3))+' + '+str(np.round(statsOutCIOFF['t_errsup'],3))+' - '+str(np.round(-statsOutCIOFF['t_errinf'],3))



In [ ]:
#-------------------------------------------------#
# Plot 
#-------------------------------------------------#
from matplotlib import pyplot as plt
from matplotlib import rc
from matplotlib import gridspec
from matplotlib.ticker import AutoMinorLocator, MultipleLocator, LogLocator
from matplotlib import transforms as mtransforms


def tSyncOverTbuoy(b,ts,tsErrSup,tsErrInf,tBuoy,tBuoySup,tBuoyInf,outfile):

    params = {'font.family'         :' serif',
              'font.style'          : 'normal',
              'font.weight'         : 'book',
              'font.size'           : 30.0,
              'axes.linewidth'      : 3,
              'lines.linewidth'     : 3,
              'xtick.labelsize'     : 26,
              'ytick.labelsize'     : 26, 
              'xtick.direction'     :'in',
              'ytick.direction'     :'in',
              'xtick.top'           : True,   # draw ticks on the top side
              'xtick.bottom'        : True,   # draw ticks on the bottom side    
              'ytick.left'          : True,   # draw ticks on the top side
              'ytick.right'         : True,   # draw ticks on the bottom side  
              'xtick.major.size'    : 8,
              'xtick.major.width'   : 3,
              'xtick.minor.size'    : 4,
              'xtick.minor.width'   : 2,
              'ytick.major.size'    : 8,
              'ytick.major.width'   : 3,
              'ytick.minor.size'    : 4,
              'ytick.minor.width'   : 2, 
              'text.usetex'         : True,
              'text.latex.unicode'  : True
               }
    plt.rcParams.update(params)

    # initialize figure
    fig = plt.figure(figsize =(12,12))
    fig.subplots_adjust(hspace=0.0)
    gs = gridspec.GridSpec(1, 1)
    plt.rc('xtick')


    # Initialize subplots
    ax1 = fig.add_subplot(gs[0])
    ax1.set_xlabel(r'Magnetic Field [$\mu$G]')
    ax1.set_ylabel(r'Source Age [Myr]')
    #ax1.set_xscale("symlog", nonposx='clip', linthreshy=0.1)
    #ax1.set_yscale("symlog", nonposy='clip', linthreshy=0.1)
    ax1.set_xscale("log", nonposx='clip')
    ax1.set_yscale("log", nonposy='clip')
    ax1.set_ylim([1e-1,2e3])
    ax1.set_xlim([0.2,5e2])
    ax1.tick_params(axis='both', which='major', pad=5)

    #POINTS
    
    # MODELS
    
#    idx_max = (np.abs(mod1_CI[0] - 5e5)).argmin()
#    idx_min = (np.abs(mod1_CI[0] - 30)).argmin()

    ax1.plot(b,ts,lw=3, marker=' ', color='red', linestyle='-',label=r't$_{\rm sync}$',alpha=1)

    ax1.fill_between(b, tsErrInf, tsErrSup, facecolor='red', alpha=0.3)
    
    ax1.plot(b,tBuoy,lw=3, marker=' ', color='blue', linestyle='--',label=r't$_{\rm buoy}$',alpha=1)
 
    ax1.fill_between(b, tBuoyInf, tBuoySup, facecolor='blue', alpha=0.3)

    legend = plt.legend(loc=3, prop={'size': 30})
    legend.get_frame().set_edgecolor('black')

    # Save figure to file
    plt.savefig(outfile,overwrite = True)


print '\n\t-----> Loaded <-----'

In [ ]:
B = np.linspace(1e-2,1e3,1e4)
BIC = 3.25*np.power(1+z,2)

stats =  syn.readCIOFFStats(dirNames[0]+'/synOut/'+names[0]+'CIOFF.stats')
    

ts = 1610.*(np.power(B,0.5))/(B*B+BIC*BIC)*np.divide(1.,np.power(stats['break']/1e3*(1.+z),0.5))
tsInf = 1610.*(np.power(B,0.5))/(B*B+BIC*BIC)*np.divide(1.,np.power((stats['break']/1e3+stats['break_sup']/1e3)*(1.+z),0.5))
tsSup = 1610.*(np.power(B,0.5))/(B*B+BIC*BIC)*np.divide(1.,np.power((stats['break']/1e3+stats['break_inf']/1e3)*(1.+z),0.5))


tBuoy = np.zeros(ts.shape)+tBuoyL
tBuoySup = np.zeros(ts.shape)+400.
tBuoyInf = np.zeros(ts.shape)+tBuoyInfL

outplot=rootDir+'tSyncOverTbuoyLobes.png'
tSyncOverTbuoy(B,ts,tsSup,tsInf,tBuoy,tBuoySup,tBuoyInf,outplot)

In [ ]:

B = np.linspace(1e-2,1e3,1e4)
BIC = 3.25*np.power(1+z,2)

stats =  syn.readCIOFFStats(dirNames[4]+'/synOut/'+names[4]+'CIOFF.stats')
    

ts = 1610.*(np.power(B,0.5))/(B*B+BIC*BIC)*np.divide(1.,np.power(stats['break']/1e3*(1.+z),0.5))
tsInf = 1610.*(np.power(B,0.5))/(B*B+BIC*BIC)*np.divide(1.,np.power((stats['break']/1e3+stats['break_sup']/1e3)*(1.+z),0.5))
tsSup = 1610.*(np.power(B,0.5))/(B*B+BIC*BIC)*np.divide(1.,np.power((stats['break']/1e3+stats['break_inf']/1e3)*(1.+z),0.5))


tBuoy = np.zeros(ts.shape)+tBuoy
tBuoySup = np.zeros(ts.shape)+600
tBuoyInf = np.zeros(ts.shape)+tBuoyInf

outplot=rootDir+'tSyncOverTbuoyOut.png'
tSyncOverTbuoy(B,ts,tsSup,tsInf,tBuoy,tBuoySup,tBuoyInf,outplot)